## **Problem Statement**

# **Object Identification Using YOLOv3**

Detect objects in the image using pretrained YOLO on COCO dataset that consists of 80 objects with labels.

## **Objective**
Detecting the objects correctly using a pretrained dataset 

### **Directories**

We are going to use 3 directories:

* **yolo-coco/**: This holds pretrained model files on the COCO dataset. Credit: *Darknet*

* **images/**: This holds 4 images for testing and evaluation 

* **output/**: This will hold output that is processed by YOLO

    Note: Open the yolo.py file to edit it for desired object detection

### **Install DarkFlow**

In [0]:
#Cloning DarkFlow (ensure that git and gitbash is intalled)
git clone https://github.com/thtrieu/darkflow

#Creating environment
conda create -n darkflow-env python=3.6

#Activating environment
source activate darkflow-env

### **Install TesorFlow, Cython, and Numpy**

In [0]:
conda install tensorflow-gpu cython numpy

### **Install OpenCV**

In [0]:
pip install opencv-contrib-python

    Note: "pip" should be installed in the system

### **Import Packages**

In [0]:
import numpy as nump
import argparse
import time
import cv2
import os

### **Add and Run Flag Arguments**


In [0]:
Parse = argparse.ArgumentParser()

#Detecting objects in the image using YOLO
Parse.add_argument("-i", "--image", required=True,
	help="Input image path")

#Loading the YOLO files to perform object detection
Parse.add_argument("-y", "--yolo", required=True,
	help="YOLO directory path")

#Filtering weak detections with minimum probability (one can use any value between 0 to 1)
Parse.add_argument("-c", "--confidence", type=float, default=0.4,
	help="Filtering weak detections with minimum probability")

#Applying non-maxima suppression with a threshold value
Parse.add_argument("-t", "--threshold", type=float, default=0.3,
	help="Applying non-maxima suppression with a threshold value")

args = vars(Parse.parse_args())

### **Load the Class Labels from Trained Model**

In [0]:
labelsPath = os.path.sep.join([args["yolo"], "coco.names"])
LABELS = open(labelsPath).read().strip().split("\n")

### **Initialize Colors that Represent Class Label**

In [0]:
nump.random.seed(40)
COLORS = nump.random.randint(0, 255, size=(len(LABELS), 3),
	dtype="uint8")

### **Derive Paths**

#### **Derive Paths to YOLO Weights**

In [0]:
weightsPath = os.path.sep.join([args["yolo"], "yolov3.weights"])

#### **Derive Paths to Model Configuration**

In [0]:
configPath = os.path.sep.join([args["yolo"], "yolov3.cfg"])

### **Load YOLO Object Detector**

In [0]:
print("[INFO] loading YOLO from disk...")
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

### **Load and Send the Image via the Network**

In [0]:
#1. Loading input image & extracting its dimension 
image = cv2.imread(args["image"])
(H, W) = image.shape[:2]


#2. Determining names of output layer from YOLO model
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

#3. Getting bounding boxes and associated probabilities by:
  #a. constructing blob from input image
blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416),
	swapRB=True, crop=False)

  #b. executing forward pass of the YOLO object detector
net.setInput(blob)
start = time.time()
layerOutputs = net.forward(ln)
end = time.time()

#4. Projecting inference time on YOLO
print("[INFO] YOLO took {:.6f} seconds".format(end - start))

### **Initialize the List**

In [0]:
boxes = [] #Bounding boxes around the object
confidences = [] #YOLO-assigned confidence value of an object
classIDs = [] #Labels of detected objects 

### **Populate Data in the List**
    Note: List includes boxes, confindence, and classIDs

In [0]:
#Looping every output
for output in layerOutputs:

	#Looping every detections
	for detection in output:

		#Extracting currently detected objects' class ID & confidence
		scores = detection[5:]
		classID = nump.argmax(scores)
		confidence = scores[classID]

		#Weeding out the weak predictions
		if confidence > args["confidence"]:
      
			#Scaling coordinates of bounding box to display them on original image
			box = detection[0:4] * nump.array([W, H, W, H])
			(centerX, centerY, width, height) = box.astype("int")
   
			#Using (x, y) coordinates to get top-left of bounding box
			x = int(centerX - (width / 2))
			y = int(centerY - (height / 2))
   
			#Updating the list
			boxes.append([x, y, int(width), int(height)])
			confidences.append(float(confidence))
			classIDs.append(classID)

### **Implement Non-Maxima Suppression**

In [0]:
idxs = cv2.dnn.NMSBoxes(boxes, confidences, args["confidence"], args["threshold"])

### **Draw NMS Boxes and Class Text on Images**

In [0]:
#Ensuring the existence of at least one detection
if len(idxs) > 0:

	#Looping the indexes
	for i in idxs.flatten():
   
		#Extracting coordinates of bounding box
		(x, y) = (boxes[i][0], boxes[i][1])
		(w, h) = (boxes[i][2], boxes[i][3])

		#Drawing rectangles in bounding box and adding random colors
		color = [int(c) for c in COLORS[classIDs[i]]]
		cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
		text = "{}: {:.4f}".format(LABELS[classIDs[i]], confidences[i])
		cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,0.5, color, 2)

### **Display an Output Image**

In [0]:
cv2.imshow("Image", image)
cv2.waitKey(0)

### **Execute YOLO in the Terminal**

In [0]:
$ python yolo.py --image images/baggage_claim.jpg --yolo yolo-coco